## Environment setup

### Library preparation

In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
from si_prefix import si_format
from tqdm import tqdm, tqdm_notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import qcodes as qc
# from qcodes import Station, load_by_run_spec, load_by_guid
# from qcodes.instrument.base import Instrument
# from qcodes.dataset.experiment_container import (Experiment,
#                                                  load_last_experiment,
#                                                  new_experiment)
# from qcodes.dataset.database import initialise_database
# from qcodes.dataset.measurements import Measurement
# from qcodes.dataset.plotting import plot_by_id, get_data_by_id, plot_dataset
# from qcodes.dataset.data_set import load_by_id
# # from qcodes.dataset.data_export import get_shaped_data_by_runidb

In [3]:
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372
from qcodes.instrument_drivers.tektronix.AWG3252_Isrc import AWG3252_Isrc
from qcodes.instrument_drivers.HP.HP34401 import HP34401
from qcodes.instrument_drivers.HP.HP34401_DL1201 import HP34401_DL1201
from qcodes.instrument_drivers.Keithley.Keithley_2600_channels import *
from qcodes.instrument_drivers.tektronix.AWG5204_Isrc import AWG5204_Isrc

from qcodes.instrument_drivers.Keysight.Keysight_B2962A_Isrc import B2962A_Isrc


from qcodes.instrument_drivers.Keithley.Keithley_6220 import Keithley_6220
from qcodes.instrument_drivers.Keithley.Keithley_6220_Isrc import Keithley_6220_Isrc

In [4]:
import sys 
sys.path.append('..\..\_jjtools')

# from Exps import *

from JJ_data_processing import read_opj_data, pbi, bpbi, show_df, update_df

from JJformulas import *
# from meas_util import *

from JJmeasurement import *

C:\Users\KelvinOX25\Anaconda3\lib\site-packages\qcodes\dataset\database.py:13: UserWarning: The module `qcodes.dataset.database` is deprecated.
Public features are available at the import of `qcodes`.
Private features are available in `qcodes.dataset.sqlite.*` modules.
  warnings.warn('The module `qcodes.dataset.database` is deprecated.\n'
C:\Users\KelvinOX25\Anaconda3\lib\site-packages\tqdm\autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### Instrument nicknames and QC database initialization

In [472]:
try:
   Instrument.close_all()
except KeyError:
   pass    
except NameError:
   pass 

K2602 =  Keithley_2600(name = 'K2602', address = "GPIB::26::INSTR" )
Flux_chA = K2602.smua.curr
Flux_chB = K2602.smub.curr
Bfield =  Flux_chA
Bfield.step = 10e-6
Bfield.inter_delay = .1



# Isrc = AWG5204_Isrc('gen', 'TCPIP0::AWG52000-XXXX::inst0::INSTR', ch = 1, R_bias = 1e9, Attn = 1, timeout = 20)

# Isrc = B2962A_Isrc(name = 'B2962A', address = "GPIB::23::INSTR" )

# Isrc = Keithley_6220(name = 'K6220', address = "GPIB::13::INSTR" )
# Isrc.set_R_Attn ( R_bias = 1, Attn = 1 )




# Isrc = AWG3252_Isrc('gen', 'TCPIP0::192.168.13.32::inst0::INSTR', 
#                     R_bias = 1e9, Attn = 10)
# Isrc.init()

Vmeter = HP34401_DL1201('meter', 'GPIB0::8::INSTR', Gain = 1e2)
Vmeter.init('fast 6')


# Isrc = Keithley_6220_Isrc(name = 'K6220', address = "GPIB::13::INSTR",
#                           Rsh = 10e6, Rb = 1e9, Vmeter = Vmeter)


Isrc = Keithley_6220_Isrc(name = 'K6220', address = "GPIB::13::INSTR",
                          Rsh = 10.041e3, Rb = 2.2e6, Vmeter = Vmeter)


station = qc.Station(Isrc, Vmeter)

print ('Instrument initialization completed.')

Connected to: Keithley Instruments Inc. 2602A (serial:1272049, firmware:2.1.5) in 0.11s


[K6220(Keithley_6220_Isrc)] Snapshot: Could not update parameter: Irange


Instrument initialization completed.


In [6]:

LS370 = Model_372(name = 'LS370 input Ch', address = 'GPIB::12::INSTR', terminator='\n')
T8 = LS370.ch08.temperature

htr = LS370.sample_heater
htr_setpt = htr.setpoint

htr_setpt.set(0.020)
htr.P.set(30)
htr.I.set(5)
htr.D.set(1)
# PIDset(30,5,1)
htr.output_range('3.16mA')

Connected to: LSCI 370 (serial:370665, firmware:04102008) in 0.10s


### Class JJmeas

In [473]:
tools = {'I' : Isrc.I,
         'V' : Vmeter.V,
         'B' : Flux_chA,
         'T': T8,
         'htr' : htr_setpt}

In [519]:
def setup():
    Isrc.I.set(0)
#     pass

def cleanup():
    Isrc.I.set(0) 
    Bfield.set(0)



In [520]:
jj = JJmeas(sample = 'D070B1N5v1', tools = tools)
jj.setup = setup
jj.cleanup = cleanup

jj.ZF   = -0.093e-3
jj.FF   = 0.130e-3

In [64]:
# jj = JJmeas(sample = 'D078N5v1', tools = tools)
# jj.setup = setup
# jj.cleanup = cleanup

In [10]:
jj.tool_status(['B', 'T'])

{'B': '-72.7 nA', 'T': '34.5 mK'}

In [186]:
jj.ZF   = -0.093e-3
jj.FF   = 0.130e-3

## Measurement

#### Pin assignment: 4ba,3ab

#### Constants

In [43]:
jj.cos_to_B(0)/1e-3

0.132

#### Iscr choise

In [224]:
Isrc.I.

<qcodes.instrument.parameter.Parameter: I at 232970288>

In [12]:
Isrc.ask_raw('SOUR:CURR:RANG?')

'2.000000E-07\n'

In [475]:
Isrc.write_raw('SOUR:CURR:RANG 1e-7')
Isrc.I.set(0.0e-9)

In [521]:
Isrc.I.set(0e-12)

In [234]:
Isrc._R_bias

1000000000.0

#### Vmeter regime

In [18]:
Vmeter.init('fast 6')

In [152]:
jj.meas_Voffset(0)

5.9592217e-05

In [162]:
jj.tools['V'].meas_Voff()
Isrc.Vmeter.Voff

5.847299300000001e-05

In [496]:
B = jj.cos_to_B(1)

print(B*1e3)

Bfield.set(B)


-0.093


### IVCs

In [379]:
jj.stabilize_I( amp = 200e-12)

In [522]:
B = jj.cos_to_B(1)

print(B*1e3)

Bfield.set(B)


jj.IVC_udu (amp = 200e-12, stp = 2.5e-12, dt = .1,  N_avg = 1, 
            label = 'kei flo box gnd + inside cab 2mhz + dc fil+ divider')

-0.093


Starting experimental run with id: 1099


1099

In [175]:
Vmeter.meas_Voff()
Isrc.Vmeter.Voff

5.9244986e-05

In [111]:
imin = .5e-12

i_list =   np.concatenate ([np.linspace(0    , 12e-12, 21), 
                            np.linspace(12e-12, imin, 21),
                            np.linspace(imin, 12e-12, 251),
                            np.linspace(12e-12,  0e-12, 21)]) 

jj.IVC_cust ( i_list, Ioff = 0, dt = 0.1, N_avg = 1, label = 'B=0m intermid steps')

Starting experimental run with id: 234


234

### Ic meas

In [345]:
B = jj.cos_to_B(0.25)
Bfield.set(B)

i_list = np.linspace(0, 30e-12, 301)

jj.IVC_cust ( i_list, Vthr = 30e-6, dt = 0.1,
             N_avg = 1, label = 'Kei Ic highRb RT cos = .25')

Starting experimental run with id: 990


990

### Time scan

In [513]:
# Bfield.set(0e-3)
jj.time_scan ( tools['I'], dur = 30, dt= 0.1)

Starting experimental run with id: 1093



1093

In [148]:
B_list = np.linspace(0e-3, .3e-3, 51)

[            jj.time_scan ( tools['V'], dur = 1, dt= 0.01)
for jj in    jj.Bscan( B_list = B_list )]

Starting experimental run with id: 372



Starting experimental run with id: 373



Starting experimental run with id: 374



Starting experimental run with id: 375



Starting experimental run with id: 376



...

Starting experimental run with id: 419



Starting experimental run with id: 420



Starting experimental run with id: 421



Starting experimental run with id: 422




[372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 421,
 422]

### IVC B

#### New syntaxis to try

In [524]:
# B_list = udu_list ( 4e-3, 0.2e-3)
B_list = np.linspace(0.0e-3, 0.15e-3, 51)
cos_list = np.linspace(1,0, 26)

[            jj.IVC_udu (amp = 100e-12, stp = 1e-12, dt = .1,  N_avg = 1, label = 'coarse B scan')
# for jj in    jj.Bscan( B_list = B_list )]
for jj in    jj.Bscan( cos_list = cos_list )]


Bfield.set(0)


Starting experimental run with id: 1101


Starting experimental run with id: 1102


Starting experimental run with id: 1103


Starting experimental run with id: 1104


Starting experimental run with id: 1105


...

Starting experimental run with id: 1122


Starting experimental run with id: 1123


Starting experimental run with id: 1124


Starting experimental run with id: 1125


Starting experimental run with id: 1126



#### Ic(B)

In [30]:
i_list = np.linspace(0, 250e-12, 501)
B_list = np.linspace(-.4e-3, .2e-3, 121)

[            jj.IVC_cust ( i_list, Vthr = 10e-6, dt = 0.1, N_avg = 1, label = 'fine Ic(B)')
for jj in    jj.Bscan( B_list = B_list )]


Bfield.set(0)

Starting experimental run with id: 689


Starting experimental run with id: 690


Starting experimental run with id: 691


Starting experimental run with id: 692


Starting experimental run with id: 693


...

Starting experimental run with id: 805


Starting experimental run with id: 806


Starting experimental run with id: 807


Starting experimental run with id: 808


Starting experimental run with id: 809



In [113]:
i_list = np.concatenate([np.linspace(0, 30e-12, 251),
                         np.linspace(30e-12, 0,  251)]) 
B_list = np.linspace(-1e-3, 1e-3, 51)

[            jj.IVC_cust ( i_list, dt = 0.1, N_avg = 1, label = 'test Ic(B)')
for jj in    jj.Bscan( B_list = B_list )]


Bfield.set(0)

Starting experimental run with id: 241


Starting experimental run with id: 242


Starting experimental run with id: 243


Starting experimental run with id: 244


Starting experimental run with id: 245


...

Starting experimental run with id: 288


Starting experimental run with id: 289


Starting experimental run with id: 290


Starting experimental run with id: 291



AttributeError: ("'Model_372' object and its delegates have no attribute 'visa_log'", "asking 'RDGK? 8\\n\\r' to <Model_372: LS370 input Ch>", 'getting LS370 input Ch_ch08_temperature')

In [174]:
# B_list = udu_list ( 4e-3, 0.2e-3)
B_list = np.linspace(0, 1.85e-3, 11)

i_list = udu_list(1e-12, 0.01e-12) -.25e-12



[            jj.IVC_cust (i_list, dt = .5, N_avg = 4)
for jj in    jj.Bscan( B_list = B_list )]

Bfield.set(0)


Starting experimental run with id: 44


Starting experimental run with id: 45


Starting experimental run with id: 46


Starting experimental run with id: 47


Starting experimental run with id: 48


...

Starting experimental run with id: 50


Starting experimental run with id: 51


Starting experimental run with id: 52


Starting experimental run with id: 53


Starting experimental run with id: 54



In [230]:
jj.ZF = 0
jj.FF = 1.85e-3
cos_list = [1,0]

[            jj.IVC_fwd (amp = 1e-12, stp = 0.1e-12, dt = .5, Ioff = -.35e-12)
for jj in    jj.Bscan( cos_list = cos_list )]

Starting experimental run with id: 63


Starting experimental run with id: 64

{ 'ids' : range(63,64+1), 'B' : np.linspace(0.00e+00,1.85e-03, 2), 'T' : 0.030, 'comm : '' }


[63, 64]

### T scan

In [210]:
5*11*6/60

5.5

In [135]:
htr.HTRset('3.16mA')

'Heater changed = 3.16mA'

In [150]:
htr.output_range('3.16mA')

In [109]:
T8.get()

0.0501275

In [ ]:
[            jj.IVC_fwd (amp = 1e-12, stp = 0.025e-12, dt = .5, N_avg = 100)
for jj in    jj.Tscan( [200e-3] )]

htr.set(0.030)

In [19]:
np.array( [50 + 25*i for i in range (11) ] )*1e-3

array([0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  , 0.225, 0.25 ,
       0.275])

In [21]:
htr_setpt.set(30e-3)

In [13]:
htr.output_range('3.16mA')

In [28]:
T_list = np.array( [50 + 25*i for i in range (11) ] )*1e-3
B_list = np.linspace(0, 1.85e-3, 11)

for jj in  jj.Tscan( T_list ):
    
    for jj in jj.Bscan(B_list):
    
        jj.IVC_udu (amp = 1e-12, stp = 0.01e-12, dt = .5, Ioff = -.5e-12)
    
    
Bfield.set(0e-3)
htr_setpt.set(30e-3)

ramping T8 to 50.0 mK...
50.9 m
50.7 m
50.7 m
50.9 m
51.0 m
50.5 m
50.5 m
50.3 m
50.6 m
50.2 m
50.2 m
50.2 m
49.9 m
49.6 m
49.8 m
50.1 m
50.0 m
49.6 m
49.7 m
49.5 m
49.7 m
49.6 m
T is set


Starting experimental run with id: 912


Starting experimental run with id: 913


Starting experimental run with id: 914


Starting experimental run with id: 915


...

Starting experimental run with id: 919


Starting experimental run with id: 920


Starting experimental run with id: 921


Starting experimental run with id: 922

{ 'ids' : range(912,922+1), 'B' : np.linspace(0.00e+00,1.85e-03, 11), 'T' : 0.050, 'comm : '' }
ramping T8 to 75.0 mK...


VisaIOError: ('VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.', "asking 'RDGK? 8\\n\\r' to <Model_372: LS370 input Ch>", 'getting LS370 input Ch_ch08_temperature')

In [ ]:
T_list = np.array( [200 + 25*i for i in range (5) ] )*1e-3
B_list = np.linspace(0, 1.85e-3, 11)

for jj in  jj.Tscan( T_list ):
    
    for jj in jj.Bscan(B_list):
    
        jj.IVC_fwd (amp = 2e-12, stp = 0.04e-12, dt = .5, Ioff = -.35e-12)
    
    
Bfield.set(0e-3)
htr_setpt.set(30e-3)

## Correcting the sample labeling

- The first device we measured at the begining of this cooldown is N8 instead of N1. We created db file "Experiments_59BBN1.db"

- The second device we measured was N2, which is also the one we revisiting. We saved the data in db file "Experiments_59BBNX.db" with runid 1-29. 

- This second device has non-zero I_offset and the IVc was extremely sensitive to the connection of current and voltage connection. For example in run id 14 & 19 we have the inner measuremnt panel box either open or closed. The real mechanism remained to be check.

- The third device we measured was N8, which is the last one we measured in this cooldown without the 100kohm chip resistor (they were shunted). We saved the data in db file "Experiments_59BBNX.db" with runid 29 and beyond. 

- This third one has again non-zero I_offset consistently resilient to various connections (the IVc doesn't change on the EM environment .